# bs

In [ ]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
head = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0'}
data = []
with requests.Session() as s:
    for page in range(1,6):
        url = f"https://www.saramin.co.kr/zf_user/jobs/list/job-category?cat_mcls=2&page={page}&page_count=1000&sort=RD"
        r = s.get(url, headers=head)
        data.append(r)
        print(page)
        if r.status_code != 200:
            break


In [ ]:
item_list = []
for r in data:
    soup = BeautifulSoup(r.text)
    box_item_list = soup.select('div.box_item')


    for box_item in box_item_list:
        temp = {}
        temp['company_name'] = box_item.select_one('.company_nm > .str_tit').text.strip()
        temp['company_sn'] = box_item.select_one('.interested_corp')['csn']
        temp['is_headhunting'] = '헤드헌팅' in box_item.select_one(".company_nm").text
        temp['job_idx'] = box_item.find('button', attrs={'rec_idx': True})['rec_idx']
        temp['job_title'] = box_item.select_one('.notification_info a.str_tit')['title']
        temp['job_link'] = box_item.select_one('.notification_info a.str_tit')['href']
        temp['job_sectors'] = [i.text for i in box_item.select('.notification_info .job_meta .job_sector span')]
        work_place_element = box_item.select_one('.recruit_info .work_place')
        temp['work_place'] = work_place_element.text if work_place_element is not None else None
        temp['career_info'] = box_item.select_one('.recruit_info .career').text
        temp['education_info'] = box_item.select_one('.recruit_info .education').text
        temp['support_date'] = box_item.select_one('.support_info .date').text
        temp['deadline'] = box_item.select_one('.support_info .deadlines').text
        item_list.append(temp)


In [ ]:

df = pd.DataFrame(item_list)
df['update'] = pd.to_datetime('now')

In [ ]:
df['update'] = pd.to_datetime('now')


In [ ]:
def process_deadline(row):
    x = row['deadline']

    word = x.split(' ')[0]

    if '일' in word:
        delta = pd.Timedelta(days=int(word.split('일')[0]))
    elif '시간' in word:
        delta = pd.Timedelta(hours=int(word.split('시간')[0]))
    elif '분' in word:
        delta = pd.Timedelta(minutes=int(word.split('분')[0]))

    if '등록' in x:
        row['resistered'] = row['update'] - delta
    elif '수정' in x:
        row['modified'] = row['update'] - delta
    else:
        pass
    
    return row

df.apply(process_deadline, axis=1)


In [ ]:
df.to_csv('saram.csv', index=False)

# base 불러오기

In [ ]:

df_old = pd.read_csv('saram.csv')

# 사명 변경된 적 있는지

In [ ]:
final_df = df[df.groupby('company_sn')['company_name'].transform('nunique') > 1].sort_values(by='job_idx')[['company_sn', 'company_name']].drop_duplicates()
final_df['name_count'] = final_df.groupby('company_sn').cumcount() + 1
final_df.pivot_table(columns='name_count', index='company_sn', values='company_name', aggfunc='first')


# 목록에 표시된 해시태그
- 하지만 각 공고마다 가지고 있는 태그 전체 내용이 아니다.

In [ ]:
df2 = df[['job_idx','job_sectors']]

In [ ]:
df2.explode('job_sectors').groupby('job_sectors').size().sort_values(ascending=False).to_csv('job_sectors.csv')

In [ ]:
df['job_link'] = df.job_link.apply(lambda x: f'https://www.saramin.co.kr{x}')

In [ ]:
df.explode('job_sectors').groupby('job_sectors').get_group('크롤링')